In [1]:
import os
import re
import json
import string
import numpy as np
import tensorflow as tf
from pprint import pprint
from tensorflow import keras
from tensorflow.keras import layers
from tokenizers import BertWordPieceTokenizer
from transformers import RobertaTokenizer, RobertaTokenizerFast, TFRobertaModel
from tensorflow import keras
import jsonlines

tf.random.set_seed(1234)
np.random.seed(1234)

# Load Data

In [2]:
seq_len = 450
max_len = 512

###################
## TRAINING DATA ##
###################
ccc_train = []

with open('../data/ccc_train.jsonl', 'r') as json_file:
    json_list = list(json_file)
for json_str in json_list:
    ccc_train.append(json.loads(json_str))

for ex in ccc_train:
    ex["text"] = " ".join(ex["text"].split())
    ex["size_text"] = " ".join(ex["size_text"].split())

print(f"Training examples from raw data: {len(ccc_train)}")
ccc_train = [a for a in ccc_train if a["size_text"].lower() in a["text"].lower()]
print(f"Training examples after removing unmatched size_text strings: {len(ccc_train)}")


######################
## SPAN TUNING DATA ##
######################
ccc_tune = []

with open('../data/ccc_span_tune.jsonl', 'r') as json_file:
    json_list = list(json_file)
for json_str in json_list:
    ccc_tune.append(json.loads(json_str))

for ex in ccc_tune:
    ex["para"] = " ".join(ex["para"].split())
    ex["size_text"] = " ".join(ex["size_text"].split())    

print(f"Span tune examples in raw data: {len(ccc_tune)}")
ccc_tune = [a for a in ccc_tune if a["size_text"].lower() in a["para"].lower()]
print(f"Span tune examples after removing unmatched size_text strings: {len(ccc_tune)}")

ccc_tune = {"data":[{"paragraphs":[{"context":qq["para"],
                             "qas":[
                                 {"answers":[{"answer_start":qq["para"].lower().index(qq["size_text"].lower()),
                                              "text":qq["size_text"]}],
                                  "question":"How many people protested?"}
                             ]}]} for qq in ccc_tune]}


##################
## TESTING DATA ##
##################
ccc_test = []

with open('../data/ccc_test_set.jsonl', 'r') as json_file:
    json_list = list(json_file)
for json_str in json_list:
    ccc_test.append(json.loads(json_str))
    
for ex in ccc_test:
    ex["text"] = " ".join(ex["text"].split())
    ex["size_text"] = " ".join(ex["size_text"].split())
    
print(f"Testing examples from raw data: {len(ccc_test)}")
ccc_test = [a for a in ccc_test if a["size_text"].lower() in a["text"].lower()]
print(f"Testing examples after removing unmatched size_text strings: {len(ccc_test)}")

ccc_test = {"data":[{"paragraphs":[{"context":" ".join(qq["text"].split()),
                             "qas":[
                                 {"answers":[{"answer_start":qq["text"].lower().index(qq["size_text"].lower()),
                                              "text":qq["size_text"]}],
                                  "question":"How many people protested?"}
                             ]}]} for qq in ccc_test]}

#####################
## VALIDATION DATA ##
#####################
ccc_valid = []

with open('../data/ccc_validation_set.jsonl', 'r') as json_file:
    json_list = list(json_file)
for json_str in json_list:
    ccc_valid.append(json.loads(json_str))
    
for ex in ccc_valid:
    ex["text"] = " ".join(ex["text"].split())
    ex["size_text"] = " ".join(ex["size_text"].split())
    
print(f"Valid examples from raw data: {len(ccc_valid)}")
ccc_valid = [a for a in ccc_valid if a["size_text"].lower() in a["text"].lower()]
print(f"Valid examples after removing unmatched size_text strings: {len(ccc_valid)}")

ccc_valid = {"data":[{"paragraphs":[{"context":" ".join(qq["text"].split()),
                             "qas":[
                                 {"answers":[{"answer_start":qq["text"].lower().index(qq["size_text"].lower()),
                                              "text":qq["size_text"]}],
                                  "question":"How many people protested?"}
                             ]}]} for qq in ccc_valid]}

########################
## SOFT-LABELED TRAIN ##
########################
soft_train = []

with open('../data/soft_labeled_train.jsonl', 'r') as json_file:
    json_list = list(json_file)
for json_str in json_list:
    soft_train.append(json.loads(json_str))
    
for ex in soft_train:
    ex["context"] = " ".join(ex["context"].split())
    ex["labeled_text"] = " ".join(ex["labeled_text"].split())
    
print(f"Soft examples from raw data: {len(soft_train)}")
soft_train = [a for a in soft_train if a["labeled_text"].lower() in a["context"].lower()]
print(f"Soft examples after removing unmatched size_text strings: {len(soft_train)}")

soft_train = {"data":[{"paragraphs":[{"context":" ".join(qq["context"].split()),
                             "qas":[
                                 {"answers":[{"answer_start":qq["context"].lower().index(qq["labeled_text"].lower()),
                                              "text":qq["labeled_text"]}],
                                  "question":"How many people protested?"}
                             ]}]} for qq in soft_train]}

#########################
## TOKENIZER AND MODEL ##
#########################

tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")


Training examples from raw data: 2694
Training examples after removing unmatched size_text strings: 2685
Span tune examples in raw data: 25
Span tune examples after removing unmatched size_text strings: 25
Testing examples from raw data: 930
Testing examples after removing unmatched size_text strings: 925
Valid examples from raw data: 200
Valid examples after removing unmatched size_text strings: 200
Soft examples from raw data: 2694
Soft examples after removing unmatched size_text strings: 2351


# Prepare Data

In [3]:
class SquadExample:
    def __init__(self, question, context, start_char_idx, answer_text, all_answers, seq_len, max_len):
        
        # Clean context, answer and question
        context = " ".join(str(context).split())
        question = " ".join(str(question).split())
        answer = " ".join(str(answer_text).split())

        self.question = str(question)
        self.context = str(context)
        self.start_char_idx = start_char_idx
        self.answer_text = str(answer)
        self.all_answers = all_answers
        self.max_len = max_len
        self.seq_len = seq_len
        self.skip_doc = False
        
        self.input_ids = None
        self.attention_mask = None
        self.token_type_ids = None
        self.start_token_idx = None
        self.end_token_idx = None
        self.skip = None
        
    def __str__(self):
        print(np.stack(self.input_ids).shape)
        print(np.stack(self.token_type_ids).shape)
        print(np.stack(self.attention_mask).shape)
        return "<SquadExample>"
        
    def __repr__(self):
        return repr({"input_ids":self.input_ids, 
                     "token_type_ids":self.token_type_ids, 
                     "attention_mask":self.attention_mask,
                     "start_token_idx":self.start_token_idx,
                     "end_token_idx":self.end_token_idx,
                     "skip":self.skip})

    def preprocess(self):
        context = self.context
        question = self.question
        answer = self.answer_text
        start_char_idx = self.start_char_idx

        # Find end character index of answer in context
        end_char_idx = start_char_idx + len(answer)
        if (end_char_idx >= len(context)) or (start_char_idx < 0):
            self.skip_doc = True
            return

        # Mark the character indexes in context that are in answer
        is_char_in_ans = [0] * len(context)
        for idx in range(start_char_idx, end_char_idx):
            is_char_in_ans[idx] = 1
            
        # Tokenize context
        tokenized_context = tokenizer(context, return_offsets_mapping=True)
        
        context_input_ids = tokenized_context.input_ids
        context_offset_mapping = tokenized_context.offset_mapping
        context_attention_mask = tokenized_context.attention_mask
        
        self.context_input_ids = context_input_ids
        self.context_offset_mapping = context_offset_mapping
        
        # Find tokens that were created from answer characters
        ans_token_idx = []
        for idx, (start, end) in enumerate(context_offset_mapping):
            if sum(is_char_in_ans[start:end]) > 0:
                ans_token_idx.append(idx)

#         if len(ans_token_idx) == 0:
#             self.skip_doc = True
#             return
        if (len(ans_token_idx) == 0):
            ans_token_idx = [-1]

        # Find start and end token index for tokens from answer
        start_token_idx = ans_token_idx[0]
        end_token_idx = ans_token_idx[-1]
        
        self.start_token_idx_master = start_token_idx
        self.end_token_idx_master = end_token_idx
        
        # Tokenize question
        tokenized_question = tokenizer("</s> "+question, return_offsets_mapping=True)
        
        ## Crop start and end tokens
        question_input_ids = tokenized_question.input_ids[1:]
        context_input_ids = context_input_ids[1:-1]

        ##
        ## SPLIT UP CONTEXT INTO MULTIPLE QUESTIONS OF max_len
        ##
        
        if seq_len >= len(context_input_ids):
            offsets = [0]
        else:
            ii = 0
            offsets = []
            while (ii+seq_len) <= len(context_input_ids):
                offsets.append(ii)
                ii = ii + round(seq_len/2)
            offsets = offsets + [len(context_input_ids)-seq_len]
        
        list_input_ids = []
        list_start_token_idx = []
        list_end_token_idx = []
        list_attention_mask = []
        list_token_type_ids = []
        list_skip = []
        
        for ii in offsets:
            subcontext_input_ids = [0]+context_input_ids[ii:(ii+seq_len)]+question_input_ids
            subcontext_start_token_idx = start_token_idx - ii
            subcontext_end_token_idx = end_token_idx - ii
            subcontext_padding = [0] * (self.max_len - len(subcontext_input_ids))
            
            subcontext_attention_mask = [1] * len(subcontext_input_ids) + subcontext_padding
            subcontext_token_type_ids = [0] + [0]*len(context_input_ids[ii:(ii+seq_len)]) + [1]*len(question_input_ids) + (np.array(subcontext_padding)+1).tolist()
            subcontext_input_ids = subcontext_input_ids + (np.array(subcontext_padding)+1).tolist()
            
            if (subcontext_start_token_idx >= 0) and (subcontext_end_token_idx < seq_len):
                skip = False
            else:
                subcontext_start_token_idx = 0
                subcontext_end_token_idx = 0
                skip = False
                
            list_input_ids.append(subcontext_input_ids)
            list_attention_mask.append(subcontext_attention_mask)
            list_token_type_ids.append(subcontext_token_type_ids)
            list_start_token_idx.append(subcontext_start_token_idx)
            list_end_token_idx.append(subcontext_end_token_idx)
            list_skip.append(skip)
        
        self.input_ids = (list_input_ids)
        self.attention_mask = (list_attention_mask)
        self.token_type_ids = (list_token_type_ids)
        self.start_token_idx = (list_start_token_idx)
        self.end_token_idx = (list_end_token_idx)
        self.skip = list_skip
        self.example_offset = offsets
        
    def train_examples(self, include_impossible=False):
        for idx, skip_ex in enumerate(self.skip):
            if include_impossible is False:
                if skip_ex is False:
                    yield  {"input_ids":self.input_ids[idx],
                            "token_type_ids":self.token_type_ids[idx],
                            "attention_mask":self.attention_mask[idx],
                            "start_token_idx":self.start_token_idx[idx],
                            "end_token_idx":self.end_token_idx[idx]}
            else:
                yield  {"input_ids":self.input_ids[idx],
                            "token_type_ids":self.token_type_ids[idx],
                            "attention_mask":self.attention_mask[idx],
                            "start_token_idx":self.start_token_idx[idx],
                            "end_token_idx":self.end_token_idx[idx]}
    
    
    def inference_from_onehot(self, pred_start, pred_end):

#         if force_answer == False:
        if (np.max(np.argmax(pred_start, axis=1)) == 0) and (np.max(np.argmax(pred_end, axis=1)) == 0):
            return("", {"all_answers":self.all_answers,
                        "start_token_pred":-1, 
                        "end_token_pred":-1, 
                        "start_char_pred":-1, 
                        "end_char_pred":-1,
                        "start_token":self.start_token_idx_master,
                        "end_token":self.end_token_idx_master,
                        "tokens_pred":set(list([])),
                        "tokens":set(list(range(self.start_token_idx_master,(self.end_token_idx_master+1))))})
        
        seq_len = min(self.seq_len, len(self.context_input_ids))
        
        pred_start_matrix = np.zeros((len(self.input_ids), len(self.context_input_ids)))
        pred_end_matrix = np.zeros((len(self.input_ids), len(self.context_input_ids)))
        
        for idx, value in enumerate(pred_start):
            offset = self.example_offset[idx]+1
            pred_start_sub = pred_start[idx][1:seq_len]
            pred_end_sub   = pred_end[idx][1:seq_len]
            pred_start_matrix[idx,(offset):(offset+seq_len-1)] = pred_start_sub
            pred_end_matrix[idx,(offset):(offset+seq_len-1)] = pred_end_sub
            
        highest_prob = np.argmax(np.max(pred_start_matrix, axis=1) + np.max(pred_end_matrix, axis=1))
        
        top_start = np.argmax(pred_start_matrix[highest_prob,:])
        top_end   = np.argmax(pred_end_matrix[highest_prob,:])
        
#         pred_start = np.max(pred_start_matrix, axis=0)
#         pred_end   = np.max(pred_end_matrix, axis=0)
        
#         top_start = np.argmax(pred_start)
#         top_end = np.argmax(pred_end)
        
        start_char = self.context_offset_mapping[top_start][0]
        end_char = self.context_offset_mapping[top_end][1]
        
        return (self.context[start_char:end_char], {"all_answers":self.all_answers,
                                                    "start_token_pred":top_start, 
                                                    "end_token_pred":top_end, 
                                                    "start_char_pred":start_char, 
                                                    "end_char_pred":end_char,
                                                    "start_token":self.start_token_idx_master,
                                                    "end_token":self.end_token_idx_master,
                                                    "tokens_pred":set(list(range(top_start,(top_end+1)))),
                                                    "tokens":set(list(range(self.start_token_idx_master,(self.end_token_idx_master+1))))})
                
    def model_inference(self, model):
        pred = model.predict([np.stack(self.input_ids),
                      np.stack(self.attention_mask),
                      np.stack(self.token_type_ids)], batch_size=8)
        pred_start = pred[0]
        pred_end   = pred[1]
        
        return self.inference_from_onehot(pred_start, pred_end)
    
    def fake_inference(self):
        pred_start_mat = []
        pred_end_mat = []
        for idx, val in enumerate(self.start_token_idx):
            pred_start = np.zeros_like(np.array(self.input_ids[idx]))
            pred_end = np.zeros_like(np.array(self.input_ids[idx]))
            if self.skip[idx] == False:
                pred_start[val] = 1.0
                pred_end[self.end_token_idx[idx]] = 1.0
            pred_start_mat.append(pred_start)
            pred_end_mat.append(pred_end)
            
        pred_start_mat = np.array(pred_start_mat)
        pred_end_mat = np.array(pred_end_mat)
        
        return self.inference_from_onehot(pred_start_mat,pred_end_mat)
        

def create_squad_examples(raw_data, seq_len, max_len):
    squad_examples = []
    for item in raw_data["data"]:
        for para in item["paragraphs"]:
            context = para["context"]
            for qa in para["qas"]:
                if len(qa["answers"]) > 0:
                    question = qa["question"]
                    answer_text = qa["answers"][0]["text"]
                    all_answers = [_["text"] for _ in qa["answers"]]
                    start_char_idx = qa["answers"][0]["answer_start"]
                    squad_eg = SquadExample(
                        question, context, start_char_idx, answer_text, all_answers, seq_len, max_len
                    )
                    squad_eg.preprocess()
                    squad_examples.append(squad_eg)
                else:
                    question = qa["question"]
                    answer_text = ""
                    all_answers = [""]
                    start_char_idx = 0
                    squad_eg = SquadExample(
                        question, context, start_char_idx, answer_text, all_answers, seq_len, max_len
                    )
                    squad_eg.preprocess()
                    squad_examples.append(squad_eg)
    return squad_examples


def create_inputs_targets(squad_examples, include_impossible=False):
    dataset_dict = {
        "input_ids": [],
        "token_type_ids": [],
        "attention_mask": [],
        "start_token_idx": [],
        "end_token_idx": [],
    }
    for item in squad_examples:
        if item.skip_doc is False:
            for example in item.train_examples(include_impossible):
                for key in dataset_dict:
                    dataset_dict[key].append(np.array(example[key]))
    for key in dataset_dict:
        dataset_dict[key] = np.array(dataset_dict[key])

    x = (
        dataset_dict["input_ids"],
        dataset_dict["token_type_ids"],
        dataset_dict["attention_mask"])
    y = (dataset_dict["start_token_idx"], dataset_dict["end_token_idx"])
    return x, y


def merge_squad_results(squad_examples, start_preds, end_preds):
    ii = 0
    tally = []
    answers = []
    f1_list = []
    for ex in squad_examples:
        if ex.skip_doc is False:
            n_sub = len(ex.skip)
            pred_out = ex.inference_from_onehot(start_preds[ii:(ii+n_sub),:], end_preds[ii:(ii+n_sub),:])
            tally.append(pred_out[0].lower() in [a.lower() for a in ex.all_answers])
            answers.append(pred_out)
            tp = len(pred_out[1]["tokens_pred"].intersection(pred_out[1]["tokens"]))
            fp = len(pred_out[1]["tokens_pred"].difference(pred_out[1]["tokens"]))
            fn = len(pred_out[1]["tokens"].difference(pred_out[1]["tokens_pred"]))
            ii = ii + n_sub
            if tp>0:
                precision = tp/(tp+fp)
                recall=tp/(tp+fn)
                f1_list.append((2*precision*recall)/(precision+recall))
            else:
                f1_list.append(0)
    return (np.mean(tally), 
            np.mean(f1_list), 
            answers)


#train_squad_examples = create_squad_examples(ccc_train, seq_len, max_len)
#x_train, y_train = create_inputs_targets(train_squad_examples)
#print(f"{len(train_squad_examples)} docs, {x_train.shape[0]} training points created.")

test_examples = create_squad_examples(ccc_test, seq_len, max_len)
x_test, y_test = create_inputs_targets(test_examples)
print(f"{len(test_examples)} docs, {x_test[0].shape} test points created.")

valid_examples = create_squad_examples(ccc_valid, seq_len, max_len)
x_valid, y_valid = create_inputs_targets(valid_examples)
print(f"{len(valid_examples)} docs, {x_valid[0].shape} valid points created.")

span_examples = create_squad_examples(ccc_tune, seq_len, max_len)
x_span, y_span = create_inputs_targets(span_examples)
print(f"{len(span_examples)} docs, {x_span[0].shape} span points created.")

soft_examples = create_squad_examples(soft_train, seq_len, max_len)
x_soft, y_soft = create_inputs_targets(soft_examples)
print(f"{len(soft_examples)} docs, {x_soft[0].shape} soft points created.")

Token indices sequence length is longer than the specified maximum sequence length for this model (1205 > 512). Running this sequence through the model will result in indexing errors


925 docs, (3306, 512) test points created.
200 docs, (725, 512) valid points created.
25 docs, (25, 512) span points created.
2351 docs, (2346, 512) soft points created.


# Train and Validate

In [4]:
strategy = tf.distribute.MirroredStrategy()
print("Number of devices: {}".format(strategy.num_replicas_in_sync))

with strategy.scope():
    model = keras.models.load_model('./roberta_base_squad2_512in')
model.summary()
keras.backend.set_value(model.optimizer.learning_rate, (5e-6))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
Number of devices: 2
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:

In [5]:
pred_start_v, pred_end_v = model.predict((x_valid[0], x_valid[1], x_valid[2]), batch_size=36, verbose=1)
valid_acc = merge_squad_results(valid_examples, pred_start_v, pred_end_v)

pred_start_t, pred_end_t = model.predict((x_test[0], x_test[1], x_test[2]), batch_size=36, verbose=1)
test_acc = merge_squad_results(test_examples, pred_start_t, pred_end_t)

print(f"Validation set accuracy (EM): {valid_acc[0]}")
print(f"Test set accuracy (EM): {test_acc[0]}")
print(f"Validation set F1: {valid_acc[1]}")
print(f"Test set F1: {test_acc[1]}")

92/92 [==============================] - 26s 285ms/step
Validation set accuracy (EM): 0.19
Test set accuracy (EM): 0.16972972972972972
Validation set F1: 0.27271624189581933
Test set F1: 0.2658847892310077


In [6]:
# validation_loss = []
# validation_accuracy = []
# batch = []
log_list = []

batch_size = 12


for ii in range(75):
    print(f"ITERATION: {ii} of 75.")
#     valid_loss = model.evaluate((x_valid[0],
#                             x_valid[1],
#                             x_valid[2]),
#                             y_valid,
#                            verbose=1,
#                            batch_size=36,
#                            return_dict=True)
#     pred_start, pred_end = model.predict((x_valid[0], x_valid[1], x_valid[2]), batch_size=36, verbose=1)
#     valid_acc = merge_squad_results(valid_examples, pred_start, pred_end)[0]
    
#     print(f"ACCURACY: {valid_acc}")
    
#     validation_loss.append(valid_loss)
#     validation_accuracy.append(valid_acc)
#     batch.append(ii*2)
    
    span_ii = np.random.choice(range(x_span[0].shape[0]), batch_size, replace=True)
    soft_ii = np.random.choice(range(x_soft[0].shape[0]), batch_size, replace=True)
    
    model.fit(
        (x_soft[0][soft_ii,:], 
         x_soft[1][soft_ii,:], 
         x_soft[2][soft_ii,:]),
        (y_soft[0][soft_ii],
         y_soft[1][soft_ii]),
        epochs=1,  # Just one epoch for the rough data
        verbose=1,
        batch_size=batch_size,  # Made this smaller. Bump up.
    )

    log = model.fit(
        (x_span[0][span_ii,:],
         x_span[1][span_ii,:],
         x_span[2][span_ii,:]),
        (y_span[0][span_ii],
         y_span[1][span_ii]),
        epochs=1,  # For demonstration, 3 epochs are recommended
        verbose=1,
        batch_size=batch_size,  # Made this smaller. Bump up.
        validation_data=((x_valid[0],
                                        x_valid[1],
                                        x_valid[2]),
                                        y_valid[0]),
                    validation_batch_size=36
    )
    
    log_list.append(log.history)
    
    
# valid_loss = model.evaluate((x_valid[0],
#                             x_valid[1],
#                             x_valid[2]),
#                             y_valid[0],
#                            verbose=1,
#                            batch_size=36,
#                            return_dict=True)
# pred_start, pred_end = model.predict((x_valid[0], x_valid[1], x_valid[2]), batch_size=36, verbose=1)
# valid_acc = merge_squad_results(valid_examples, pred_start, pred_end)[0]

# validation_loss.append(valid_loss)
# validation_accuracy.append(valid_acc)
# batch.append(ii*2)


ITERATION: 0 of 75.
INFO:tensorflow:batch_all_reduce: 198 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1').
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1').
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1').
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/t

1/1 [==============================] - 6s 6s/step - activation_5_loss: 0.0809 - loss: 0.5013 - activation_4_loss: 0.4205 - val_activation_5_loss: 3.7336 - val_loss: 5.7366 - val_activation_4_loss: 2.0030
ITERATION: 15 of 75.
1/1 [==============================] - 6s 6s/step - activation_5_loss: 0.1479 - loss: 0.6340 - activation_4_loss: 0.4861 - val_activation_5_loss: 3.7545 - val_loss: 5.7791 - val_activation_4_loss: 2.0246
ITERATION: 16 of 75.
1/1 [==============================] - 6s 6s/step - activation_5_loss: 0.1135 - loss: 0.5482 - activation_4_loss: 0.4346 - val_activation_5_loss: 3.7801 - val_loss: 5.8293 - val_activation_4_loss: 2.0492
ITERATION: 17 of 75.
1/1 [==============================] - 6s 6s/step - activation_5_loss: 0.0688 - loss: 0.2828 - activation_4_loss: 0.2140 - val_activation_5_loss: 3.8018 - val_loss: 5.8716 - val_activation_4_loss: 2.0698
ITERATION: 18 of 75.
1/1 [==============================] - 6s 6s/step - activation_5_loss: 0.0583 - loss: 0.1872 - activ

1/1 [==============================] - 6s 6s/step - activation_5_loss: 0.0513 - loss: 0.3761 - activation_4_loss: 0.3248 - val_activation_5_loss: 4.3522 - val_loss: 6.9737 - val_activation_4_loss: 2.6215
ITERATION: 39 of 75.
1/1 [==============================] - 6s 6s/step - activation_5_loss: 0.0746 - loss: 0.2012 - activation_4_loss: 0.1266 - val_activation_5_loss: 4.4035 - val_loss: 7.0701 - val_activation_4_loss: 2.6666
ITERATION: 40 of 75.
1/1 [==============================] - 6s 6s/step - activation_5_loss: 0.1581 - loss: 0.3720 - activation_4_loss: 0.2139 - val_activation_5_loss: 4.4495 - val_loss: 7.1539 - val_activation_4_loss: 2.7044
ITERATION: 41 of 75.
1/1 [==============================] - 6s 6s/step - activation_5_loss: 0.0749 - loss: 0.2892 - activation_4_loss: 0.2143 - val_activation_5_loss: 4.4944 - val_loss: 7.2390 - val_activation_4_loss: 2.7446
ITERATION: 42 of 75.
1/1 [==============================] - 6s 6s/step - activation_5_loss: 0.0425 - loss: 0.2365 - activ

1/1 [==============================] - 6s 6s/step - activation_5_loss: 0.0119 - loss: 0.1226 - activation_4_loss: 0.1107 - val_activation_5_loss: 4.9910 - val_loss: 8.0155 - val_activation_4_loss: 3.0245
ITERATION: 62 of 75.
1/1 [==============================] - 6s 6s/step - activation_5_loss: 0.0448 - loss: 0.2668 - activation_4_loss: 0.2220 - val_activation_5_loss: 5.0132 - val_loss: 8.0518 - val_activation_4_loss: 3.0386
ITERATION: 63 of 75.
1/1 [==============================] - 6s 6s/step - activation_5_loss: 0.0485 - loss: 0.1794 - activation_4_loss: 0.1309 - val_activation_5_loss: 5.0382 - val_loss: 8.0950 - val_activation_4_loss: 3.0568
ITERATION: 64 of 75.
1/1 [==============================] - 6s 6s/step - activation_5_loss: 0.0627 - loss: 0.2166 - activation_4_loss: 0.1539 - val_activation_5_loss: 5.0617 - val_loss: 8.1358 - val_activation_4_loss: 3.0741
ITERATION: 65 of 75.
1/1 [==============================] - 6s 6s/step - activation_5_loss: 0.0308 - loss: 0.1012 - activ

In [7]:
import pandas as pd

stats = pd.DataFrame(log_list)
stats.to_csv("results/roberta_base_squad2_512in_cccspans_softspans_75ii.csv")
model.save("roberta_base_squad2_512in_cccspans_softspans_75ii")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: roberta_base_squad2_512in_cccspans_softspans_75ii/assets


In [9]:
pred_start_v, pred_end_v = model.predict((x_valid[0], x_valid[1], x_valid[2]), batch_size=36, verbose=1)
valid_acc = merge_squad_results(valid_examples, pred_start_v, pred_end_v)

pred_start_t, pred_end_t = model.predict((x_test[0], x_test[1], x_test[2]), batch_size=36, verbose=1)
test_acc = merge_squad_results(test_examples, pred_start_t, pred_end_t)

print(f"Validation set accuracy (EM): {valid_acc[0]}")
print(f"Test set accuracy (EM): {test_acc[0]}")
print(f"Validation set F1: {valid_acc[1]}")
print(f"Test set F1: {test_acc[1]}")

pd.DataFrame([{"model":"roberta_base_squad2_512in_cccspans_softspans_75ii",
              "f1_valid":valid_acc[1],"f1_test":test_acc[1],
              "em_valid":valid_acc[0],"em_test":test_acc[0]}]).to_csv("results/roberta_f1v_f1t_emv_emt.csv",mode="a",header=False)

92/92 [==============================] - 27s 293ms/step
Validation set accuracy (EM): 0.655
Test set accuracy (EM): 0.6248648648648648
Validation set F1: 0.6193650793650793
Test set F1: 0.6099081696441716
